In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/three-models/densenet_model.h5
/kaggle/input/three-models/mobilenet_model.h5
/kaggle/input/three-models/efficientnet_model.h5
/kaggle/input/dogs-vs-cats-redux-kernels-edition/sample_submission.csv
/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip
/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip


In [2]:
import zipfile
import os

# Unzip train and test data
with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/train.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/train")

with zipfile.ZipFile("/kaggle/input/dogs-vs-cats-redux-kernels-edition/test.zip", "r") as zip_ref:
    zip_ref.extractall("/kaggle/working/test")

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping

2025-04-21 19:43:37.130243: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745264617.343448      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745264617.402299      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
# ================================
# STEP 1: Imports and Setup
# ================================
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2, DenseNet121, EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import Input

# ================================
# STEP 2: Paths and Preprocessing
# ================================
train_dir = "/kaggle/working/train/train"
test_dir = "/kaggle/working/test/test"
IMG_SIZE = 224
BATCH_SIZE = 32

# Labels from filenames
filenames = os.listdir(train_dir)
labels = ['dog' if 'dog' in fname else 'cat' for fname in filenames]
df = pd.DataFrame({'filename': filenames, 'class': labels})

# Split into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.1, stratify=df['class'], random_state=42)

In [5]:
# ================================
# STEP 3: Image Generators
# ================================
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, rotation_range=15, zoom_range=0.2)
val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_dataframe(train_df, train_dir, x_col='filename', y_col='class',
                                           target_size=(IMG_SIZE, IMG_SIZE), class_mode='binary', batch_size=BATCH_SIZE)

val_data = val_gen.flow_from_dataframe(val_df, train_dir, x_col='filename', y_col='class',
                                       target_size=(IMG_SIZE, IMG_SIZE), class_mode='binary', batch_size=BATCH_SIZE)

test_df = pd.DataFrame({'filename': os.listdir(test_dir)})
test_data = test_gen.flow_from_dataframe(test_df, test_dir, x_col='filename', class_mode=None,
                                         target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE, shuffle=False)

Found 22500 validated image filenames belonging to 2 classes.
Found 2500 validated image filenames belonging to 2 classes.
Found 12500 validated image filenames.


In [6]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

# STEP 1: Load all 3 models
mobilenet_model = load_model('/kaggle/input/three-models/mobilenet_model.h5')
print("MobileNetV2 loaded")

densenet_model = load_model('/kaggle/input/three-models/densenet_model.h5')
print("DenseNet121 loaded")

efficientnet_model = load_model('/kaggle/input/three-models/efficientnet_model.h5')
print("EfficientNetB0 loaded")


I0000 00:00:1745264631.651752      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1745264631.652442      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


MobileNetV2 loaded
DenseNet121 loaded
EfficientNetB0 loaded


In [7]:
# STEP 2: Predict on test data using each model
mobilenet_preds = mobilenet_model.predict(test_data)
densenet_preds = densenet_model.predict(test_data)
efficientnet_preds = efficientnet_model.predict(test_data)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1745264640.462600      62 service.cc:148] XLA service 0x7cebdc002150 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745264640.463452      62 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1745264640.463474      62 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1745264640.945945      62 cuda_dnn.cc:529] Loaded cuDNN version 90300


  5/391 ━━━━━━━━━━━━━━━━━━━━ 17s 45ms/step

I0000 00:00:1745264644.724592      62 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


391/391 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step
391/391 ━━━━━━━━━━━━━━━━━━━━ 53s 98ms/step
391/391 ━━━━━━━━━━━━━━━━━━━━ 35s 73ms/step


In [8]:
# STEP 3: Ensemble (average the predictions)
ensemble_preds = (mobilenet_preds + densenet_preds + efficientnet_preds) / 3
ensemble_preds_labels = (ensemble_preds > 0.5).astype(int).ravel()


In [9]:
# STEP 4: Format submission
test_df['id'] = test_df['filename'].apply(lambda x: int(x.split('.')[0]))

submission = pd.DataFrame({
    'id': test_df['id'],
    'label': ensemble_preds_labels
}).sort_values('id')

submission.to_csv('submission.csv', index=False)
print("📄 Ensemble submission saved as 'submission.csv'")


📄 Ensemble submission saved as 'submission.csv'
